In [1]:
from nba_api.stats.endpoints import playercareerstats
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.library.parameters import SeasonTypePlayoffs

import json


In [2]:
def json_to_file(f_name, sometext):
    json_object = json.dumps(sometext, indent=4)
    with open(f'data/{f_name}.json', "w") as outfile:
        outfile.write(json_object)


In [3]:
nba_teams = teams.get_teams()

json_to_file("nba_teams",nba_teams)

In [4]:
team_abb_collection = []
for i in teams.get_teams():
    team_abb_collection.append(i['abbreviation'])

# team_abb_collection

In [5]:
# Select the dictionary for the Team (BOS, LAL, IND), which contains their team ID 
team_abb = 'DAL'
team_select = [team for team in nba_teams if team['abbreviation'] == team_abb][0]
team_select

{'id': 1610612742,
 'full_name': 'Dallas Mavericks',
 'abbreviation': 'DAL',
 'nickname': 'Mavericks',
 'city': 'Dallas',
 'state': 'Texas',
 'year_founded': 1980}

In [6]:
f'team_id: {team_select["id"]}'

'team_id: 1610612742'

In [49]:
# export player to file
all_player_list = players.get_players()
json_to_file("all_player",all_player_list)

In [53]:
player_id_map = {}
for i in all_player_list:
    # print(i)
    player_id_map[i['id']] = i['full_name']
    # break
# player_id_map

In [8]:
# Search for player with name
player_name = "Bol"
player_result = []
for i in all_player_list:
    if i['first_name'] == player_name:
        player_result.append(i)
    elif i['last_name'] == player_name:
        player_result.append(i)
    
# print(player_result)

[{'id': 1629626, 'full_name': 'Bol Bol', 'first_name': 'Bol', 'last_name': 'Bol', 'is_active': True}, {'id': 76195, 'full_name': 'Manute Bol', 'first_name': 'Manute', 'last_name': 'Bol', 'is_active': False}]


In [9]:
# Nikola Jokić
career = playercareerstats.PlayerCareerStats(player_id='203999') 


In [54]:
# career.get_json()
test_1 = career.get_data_frames()[0]

test_map = {203999: "Joker"}

# test_1['player_name'] = test_1['PLAYER_ID']
# test_1 = test_1.assign(assignTest = 'Cat')
test_1['PLAYER_NAME'] = test_1['PLAYER_ID'].map(player_id_map)

test_1.to_csv('data/palyer_Stat.csv', sep=',',  index=False)

test_1


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLAYER_NAME
0,203999,2015-16,00,1610612743,DEN,21.0,80,55,1733.0,307,...,181,379,560,189,79,50,104,208,796,Nikola Jokic
1,203999,2016-17,00,1610612743,DEN,22.0,73,59,2038.0,494,...,212,506,718,359,61,55,171,214,1221,Nikola Jokic
2,203999,2017-18,00,1610612743,DEN,23.0,75,73,2443.0,504,...,195,608,803,458,90,61,210,212,1385,Nikola Jokic
3,203999,2018-19,00,1610612743,DEN,24.0,80,80,2504.0,616,...,228,637,865,580,108,55,248,228,1604,Nikola Jokic
4,203999,2019-20,00,1610612743,DEN,25.0,73,73,2335.0,565,...,166,545,711,512,85,44,226,222,1456,Nikola Jokic
5,203999,2020-21,00,1610612743,DEN,26.0,72,72,2488.0,732,...,205,575,780,599,95,48,222,192,1898,Nikola Jokic
6,203999,2021-22,00,1610612743,DEN,27.0,74,74,2476.0,764,...,206,813,1019,584,109,63,281,191,2004,Nikola Jokic
7,203999,2022-23,00,1610612743,DEN,28.0,69,69,2323.0,646,...,167,650,817,678,87,47,247,174,1690,Nikola Jokic
8,203999,2023-24,00,1610612743,DEN,29.0,79,79,2737.0,822,...,223,753,976,708,108,68,237,194,2085,Nikola Jokic


In [11]:
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_select['id'],
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular
                            # season_type_nullable=SeasonTypePlayoffs.playoffs
                                              )  

games_dict = gamefinder.get_normalized_dict()
# print(SeasonTypePlayoffs.playoffs)
gamefinder


In [12]:
match_up_team = "BOS"
matchup_result = []
for i in games_dict['LeagueGameFinderResults']:
    if match_up_team in i['MATCHUP']:
    # if match_up_team.MATCH.str.contsins('MIN:
        matchup_result.append(i)

json_to_file("match_result",matchup_result)

        

In [13]:
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

# print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')
print(game)

{'SEASON_ID': '22023', 'TEAM_ID': 1610612742, 'TEAM_ABBREVIATION': 'DAL', 'TEAM_NAME': 'Dallas Mavericks', 'GAME_ID': '0022301196', 'GAME_DATE': '2024-04-14', 'MATCHUP': 'DAL @ OKC', 'WL': 'L', 'MIN': 241, 'PTS': 86, 'FGM': 32, 'FGA': 97, 'FG_PCT': 0.33, 'FG3M': 8, 'FG3A': 39, 'FG3_PCT': 0.205, 'FTM': 14, 'FTA': 17, 'FT_PCT': 0.824, 'OREB': 12, 'DREB': 29, 'REB': 41, 'AST': 19, 'STL': 8, 'BLK': 2, 'TOV': 14, 'PF': 15, 'PLUS_MINUS': -49.0}


In [14]:
# Get Playbyplay
from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2('0022301196')
pbp = pbp.get_data_frames()[0]
# pbp.head()

In [15]:
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022301196') 

# box.away_team_player_stats.get_dict()  

# Data Sets
# box.game.get_dict()                  #equal to box.get_dict()['game']
#box.arena.get_dict()                  #equal to box.get_dict()['game']['arena']
#box.away_team.get_dict()              #equal to box.get_dict()['game']['awayTeam']
#box.away_team_player_stats.get_dict() #equal to box.get_dict()['game']['awayTeam']['players']
#box.away_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.home_team.get_dict()              #equal to box.get_dict()['game']['homeTeam']
#box.home_team_player_stats.get_dict() #equal to box.get_dict()['game']['homeTeam']['players']
#box.home_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.game_details.get_dict()           #equal to box.get_dict()['game'] scrubbed of all other dictionaries
#box.officials.get_dict()              #equal to box.get_dict()['game']['officials']

In [17]:
# gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable="1610612742")

games = gamefinder.get_data_frames()[0]


total_game = games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count().loc['2015':]
total_game

,GAME_ID
SEASON_ID,
2023,82
